In [1]:
!pip install fuzzywuzzy

In [164]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import numpy as np
from tqdm import tqdm
from time import sleep
import  csv
import nltk
from nltk.stem import WordNetLemmatizer
import string
from nltk.corpus import stopwords


In [165]:
########Stackoverflow approach:

patterns="""
    NP: {<JJ>*<NN*>+}
    {<JJ>*<NN*><CC>*<NN*>+}
    {<NP><CC><NP>}
    {<RB><JJ>*<NN*>+}
    """

NPChunker = nltk.RegexpParser(patterns)

def prepare_text(input):
    sentences = nltk.sent_tokenize(input)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
#     print(sentences)
    sentences = [[word for word in sent if word!='i'] for sent in sentences]
#     print(sentences)
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    sentences = [NPChunker.parse(sent) for sent in sentences]
    return sentences


def parsed_text_to_NP(sentences):
    nps = []
    for sent in sentences:
        tree = NPChunker.parse(sent)
#         print(tree)
        for subtree in tree.subtrees():
            if subtree.label() == 'NP':
                t = subtree
                t = ' '.join(word for word, tag in t.leaves())
                nps.append(t)
    return nps


def sent_parse(input):
    sentences = prepare_text(input)
    nps = parsed_text_to_NP(sentences)
    return nps




def get_clean_sentence(text):
    """
    Converting text to lower case as in, converting "Hello" to  "hello" or "HELLO" to "hello".
    """
    wordnet_lemmatizer = WordNetLemmatizer()
    for c in string.punctuation:
        text = text.replace(c,' ')
    tokens = nltk.word_tokenize(text)
    words = [word for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w.lower() for w in words if not w in stop_words]
    return ' '.join(words)

In [179]:
def clean_dish_name(text):
#     text = 'aachari murgh'
    cleaned_review = get_clean_sentence(text)
    get_entities = sent_parse(cleaned_review)
    if len(get_entities) > 0:
#         print(get_entities)
#         print(get_entities)
        return get_entities[0]
    return None
# print(get_entities)

In [255]:
word_dict = {}
yelp_word_list = pd.read_csv('food_file.txt', sep='\n',header=None,encoding='utf-8')
print(yelp_word_list.shape)
yelp_word_list = [word[0] for word in yelp_word_list.values]
yelp_word_list = np.asarray([str(word).lower() for word in yelp_word_list])
yelp_word_list = np.asarray([word.replace("'",'') for word in yelp_word_list])
yelp_word_list = np.unique(yelp_word_list)
yelp_word_list.shape

(42459, 1)


(42354,)

In [256]:
%%time
vf = np.vectorize(clean_dish_name)
yelp_word_list = vf(yelp_word_list)
yelp_word_list.sort()
yelp_word_list.shape
# np.asarray([clean_dish_name(dish) for dish in word_list])

Wall time: 1min 10s


(42354,)

In [264]:
yelp_word_list = np.asarray([w for w in yelp_word_list if w != 'None'])
yelp_word_list = np.unique(yelp_word_list)
for word in unwanted_words:
    try:
        yelp_word_list.remove(word)
    except:
        pass
yelp_word_list.shape

(38844,)

In [267]:
yelp_df = pd.DataFrame(yelp_word_list)
yelp_df.to_csv('yelp_foodset.csv',index=False,header=None)

In [240]:
# %%time
# query = 'country gentleman corn'
# print("List of ratios: ")
# print(process.extract(query, word_list, scorer=fuzz.partial_token_set_ratio), '\n')
# # print("Best among the above list: ", process.extractOne(query, word_list))

In [241]:
##Get simple list of dishes - one word only

# dict_words = {}
# for word in new_word_list:
#     for w in word.split(' '):
#         dict_words[w] = dict_words.get(w,0)+1

In [208]:
# food_set = [(dict_words[x],x) for x in dict_words.keys()]
# food_set.sort(reverse=True)
# # print(food_set)
# simple_list = pd.DataFrame([x[1] for x in food_set])

In [212]:
# simple_list.shape

(11979, 1)

In [242]:
# np.savetxt('simple_list.txt',simple_list)
# print(simple_list)
# simple_list.to_csv('simple_list.csv',index=False,header=False)


##Simple approach to reduce food list

In [247]:
###Refine simple_edited list using unwanted word list

word_list = pd.read_csv('simple_list_edited.csv', sep='\n',header=None,encoding='utf-8')
unwanted_words = pd.read_csv('unwanted_words.csv', sep='\n',header=None,encoding='utf-8')

word_list = [word[0] for word in word_list.values]
unwanted_words = [word[0] for word in unwanted_words.values]

print(len(word_list))
print(len(unwanted_words))

for word in unwanted_words:
    try:
        word_list.remove(word)
    except:
        pass
    
print(len(word_list))
print(len(unwanted_words))

948
203
948
203


In [244]:
word_list = pd.DataFrame(word_list)
word_list.to_csv('simple_list_edited.csv', header=None, index=False)

In [311]:
dict_words[word_list[1]]

939

In [281]:
print(yelp_word_list[:5])
print(word_list[:5])

['aachari murgh' 'aaloocholay' 'aardvark sauce' 'abacaxai mai tai'
 'abalone style calamari']
['chicken', 'salad', 'cream', 'chocolate', 'beef']


In [308]:
%%time
## Using fuzzy matches of words from simple list to get more dishes from new_word_list
for dish in tqdm(word_list[1:2]):
    print(dish, ': ' ,process.extract(dish, yelp_word_list, scorer=fuzz.token_set_ratio, limit=1000), '\n')


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.41s/it]

salad :  [('additional large salad', 100), ('ahi poki salad', 100), ('ahi salad', 100), ('ahi tuna avocado salad', 100), ('ahi tuna entree salad', 100), ('ahi tuna poke salad', 100), ('ahi tuna salad', 100), ('al pastor salad', 100), ('american salad', 100), ('anchovy potato salad', 100), ('antipasta salad', 100), ('antipasti salad', 100), ('antipasto salad', 100), ('antipato salad', 100), ('anything everything salad', 100), ('anzu house salad', 100), ('apple avocado salad', 100), ('apple fennel salad', 100), ('apple gorgonzola salad', 100), ('apple salad', 100), ('arabic salad', 100), ('argentina salad', 100), ('arizona salad', 100), ('artichoke salad', 100), ('arugala salad', 100), ('arugula fennel salad', 100), ('arugula salad', 100), ('arugula salad focaccia', 100), ('arugula salad pizza', 100), ('arugula salad w', 100), ('asbury spring salad', 100), ('asian chicken salad', 100), ('asian salad', 100), ('asian salmon salad', 100), ('asian sesame chicken salad', 100), ('asian sesame 

In [327]:
%%time
extended_list = set()
## Using fuzzy matches of words from simple list to get more dishes from new_word_list
for dish in tqdm(word_list):
    matches = process.extract(dish, yelp_word_list, scorer=fuzz.token_set_ratio, limit=dict_words[dish])
    for match in matches:
        if match[1] > 90:
            extended_list.add(match[0])


100%|████████████████████████████████████████████████████████████████████████████████| 948/948 [42:02<00:00,  2.66s/it]

Wall time: 42min 2s


In [329]:
print(len(extended_list))
for word in word_list:
    extended_list.add(word)
print(len(word_list))
print(len(extended_list))

27985
948
27985


In [330]:
extended_list_df = pd.DataFrame(extended_list)
extended_list_df.to_csv('extended_list.csv', header=None, index=False)

In [371]:
def updateRating(old_tuple,rating):
    old_rating, old_count = old_tuple
    new_rating = (old_rating*old_count + rating)/(old_count+1)
    return (new_rating, old_count+1)

def getDishes3(business_ids, reviews, ratings):
    result = []
    global matching_cache, food_count, food_to_biz
    for business_id, text, rating in tqdm(zip(business_ids, reviews, ratings)):
        cleaned_review = get_clean_sentence(text)
        get_entities = sent_parse(text) + sent_parse(cleaned_review)
        get_unique_entities = set(get_entities)
        print(get_unique_entities)
        dishes_in_review = set([])
        for entity in get_unique_entities:
            dish = None
            if matching_cache.get(entity,0) == 0:
                match = process.extractOne(entity, word_list, scorer=fuzz.token_set_ratio)
                if match[1]>=70:
                    matching_cache[entity] = match[0]
                else:
                    matching_cache[entity] = None

            if matching_cache[entity] != None:
                dish = matching_cache[entity]
            
            if dish != None:
                dishes_in_review.add(dish)
                
        for dish in dishes_in_review:
            temp = food_to_biz.setdefault(dish,{})
            temp[business_id] = updateRating(temp.setdefault(business_id,(0,0)),rating)
            food_to_biz[dish] = temp
            food_count[dish] = updateRating(food_count.setdefault(dish,(0,0)),rating)

            
        result.append(dishes_in_review)
    return result

In [372]:
%%time
###Ran with extended_list
text = "I love their salads and soups! Yes, Paradise is known for their cookies.  I've actually bought over 20 boxes one time for an individual's birthday plus a big cookie.  The staff is really helpful and always friendly.  Personally,  I'm over the cookies and chips but the Chocolate Chip Banana muffin heated is so good anytime of the day!  I'm not a fan of their French Onion but I usually mix the Chicken and Wild Rice with the Broccoli and Cheese.  A cup is under $5 and it's pretty filling with the roll.  Their newest Spinach salad with bacon, mushrooms, and egg is very tasty -sweet mainly and filling.Lunch time can be crazy busy with business professionals and tourists who cannot decide what to get.  Wi-Fi is free for two hours. BEWARE: On Sundays, they do clean out the drains, talk about smelly!"

matching_cache = {}
food_to_biz = {}
food_count = {}
getDishes3([123],[text],[4])

0it [00:00, ?it/s]

{'egg', 'individual', 'good anytime', 'tasty sweet', 'crazy busy business', 'filling.Lunch time', 'muffin', 'fan', 'spinach', 'french onion', 'business', 'big cookie', 'individual birthday plus big cookie', 'bacon', 'paradise', 'cup', 'talk', 'BEWARE', 'lunch time', 'roll', 'time', 'chicken wild rice broccoli', 'birthday', 'individual birthday', 'day', 'salad bacon', 'chocolate chip banana muffin', 'staff', 'tasty -sweet', 'day fan', 'Spinach salad'}


1it [00:01,  1.88s/it]

Wall time: 1.88 s


[{'bacon',
  'bear',
  'chicken',
  'chocolate',
  'cookie',
  'daddy',
  'egg',
  'flan',
  'guinness',
  'lime',
  'muffin',
  'onion',
  'paradise',
  'roll',
  'salad',
  'spinach',
  'sweet',
  'tako'}]

In [358]:
%%time
###Ran with word_list

text = "I love their salads and soups! Yes, Paradise is known for their cookies.  I've actually bought over 20 boxes one time for an individual's birthday plus a big cookie.  The staff is really helpful and always friendly.  Personally,  I'm over the cookies and chips but the Chocolate Chip Banana muffin heated is so good anytime of the day!  I'm not a fan of their French Onion but I usually mix the Chicken and Wild Rice with the Broccoli and Cheese.  A cup is under $5 and it's pretty filling with the roll.  Their newest Spinach salad with bacon, mushrooms, and egg is very tasty -sweet mainly and filling.Lunch time can be crazy busy with business professionals and tourists who cannot decide what to get.  Wi-Fi is free for two hours. BEWARE: On Sundays, they do clean out the drains, talk about smelly!"

matching_cache = {}
food_to_biz = {}
food_count = {}
getDishes3([123],[text],[4])

0it [00:00, ?it/s]

{'lunch time', 'spinach', 'french onion', 'tasty sweet', 'crazy busy business', 'big cookie', 'time', 'salad bacon', 'chicken wild rice broccoli', 'chocolate chip banana muffin', 'individual birthday plus big cookie', 'staff', 'paradise', 'cup', 'day fan', 'individual birthday'}


1it [00:00,  1.03it/s]

Wall time: 978 ms


[{'chicken',
  'chocolate',
  'cookie',
  'lime',
  'onion',
  'paradise',
  'salad',
  'spinach',
  'sweet'}]

In [356]:
%%time
###Ran with word_list and cleaned

text = "I love their salads and soups! Yes, Paradise is known for their cookies.  I've actually bought over 20 boxes one time for an individual's birthday plus a big cookie.  The staff is really helpful and always friendly.  Personally,  I'm over the cookies and chips but the Chocolate Chip Banana muffin heated is so good anytime of the day!  I'm not a fan of their French Onion but I usually mix the Chicken and Wild Rice with the Broccoli and Cheese.  A cup is under $5 and it's pretty filling with the roll.  Their newest Spinach salad with bacon, mushrooms, and egg is very tasty -sweet mainly and filling.Lunch time can be crazy busy with business professionals and tourists who cannot decide what to get.  Wi-Fi is free for two hours. BEWARE: On Sundays, they do clean out the drains, talk about smelly!"

matching_cache = {}
food_to_biz = {}
food_count = {}
getDishes3([123],[text],[4])

0it [00:00, ?it/s]

{'roll', 'day', 'egg', 'individual', 'fan', 'talk', 'good anytime', 'business', 'big cookie', 'time', 'filling.Lunch time', 'staff', 'bacon', 'muffin', 'tasty -sweet', 'cup', 'birthday', 'Spinach salad', 'BEWARE'}


1it [00:01,  1.00s/it]

Wall time: 1 s


[{'bacon',
  'bear',
  'cookie',
  'daddy',
  'egg',
  'flan',
  'guinness',
  'lime',
  'muffin',
  'roll',
  'salad',
  'sweet',
  'tako'}]

In [362]:
%%time
###Ran with word_list and cleaned, no scorer

text = "I love their salads and soups! Yes, Paradise is known for their cookies.  I've actually bought over 20 boxes one time for an individual's birthday plus a big cookie.  The staff is really helpful and always friendly.  Personally,  I'm over the cookies and chips but the Chocolate Chip Banana muffin heated is so good anytime of the day!  I'm not a fan of their French Onion but I usually mix the Chicken and Wild Rice with the Broccoli and Cheese.  A cup is under $5 and it's pretty filling with the roll.  Their newest Spinach salad with bacon, mushrooms, and egg is very tasty -sweet mainly and filling.Lunch time can be crazy busy with business professionals and tourists who cannot decide what to get.  Wi-Fi is free for two hours. BEWARE: On Sundays, they do clean out the drains, talk about smelly!"

matching_cache = {}
food_to_biz = {}
food_count = {}
getDishes3([123],[text],[4])

0it [00:00, ?it/s]

{'lunch time', 'spinach', 'french onion', 'tasty sweet', 'crazy busy business', 'big cookie', 'time', 'salad bacon', 'chicken wild rice broccoli', 'chocolate chip banana muffin', 'individual birthday plus big cookie', 'staff', 'paradise', 'cup', 'day fan', 'individual birthday'}


1it [00:35, 35.48s/it]

Wall time: 35.5 s


[{'big cookie',
  'brunch time',
  'chicken broccoli',
  'chocolate chip muffin',
  'dan dan',
  'french onion',
  'lime',
  'paradise',
  'slab bacon',
  'spinach',
  'taste sweet'}]

In [279]:
# ####Older approach - self loop, very expensive

# temp = []
# for word in tqdm(word_list):
# #     sleep(0.01)
#     temp.append(process.extract(word, word_list, scorer=fuzz.partial_ratio))
    
# with open("out.csv","w", newline='') as f:
#     wr = csv.writer(f,delimiter=',')
#     wr.writerows(temp)

In [ ]:
# food_dict = []
# print(len(food_dict))
# for i in range(len(temp[:2])):
#     matches = temp[i]
# #     print(matches)
#     matches.sort(key = lambda x:len(x[0]))
# #     print(temp[i])
#     for match in matches:
#         if match[1] >=100:
#             food_dict.append(match[0])
#             break
# print((set(food_dict)))

In [130]:
food_dict = []
print(len(food_dict))
for matches in temp:
    for match in matches[1:]:
#         print(match)
        if match[1] >=90:
            food_dict.append(match[0])

food_dict = set(food_dict)
print(len(food_dict))

0
4108


In [131]:
food_dict = list(food_dict)
# food_dict.sort(key = lambda x: (x,len(x)))

df = pd.DataFrame(food_dict)
df.to_csv('food_dict_90_percent_matches.csv', index=False)

In [101]:
test = 'chocolate chip banana muffin'
#Standard
print(process.extract(test, food_dict))

[('banana', 90), ('chocolat', 90), ('chocolate', 90), ('banana fritters', 86), ('banana ice cream', 86)]


In [98]:
#Best
print(process.extract(test, food_dict, scorer=fuzz.partial_ratio))

[('banana', 100), ('chocolat', 100), ('chocolate', 100), ('muffins', 92), ('bananas', 86)]


In [102]:
#Worst
print(process.extractOne(test, food_dict, scorer=fuzz.token_sort_ratio))

('chocolate pudding', 58)


In [103]:
#Better
print(process.extractOne(test, food_dict, scorer=fuzz.token_set_ratio))

('banana', 100)


In [106]:
###Yelp photo json
photo = pd.read_json('photos.json',lines=True)

In [ ]:
photo.drop(['business_id','photo_id'], axis=1, inplace=True)

In [128]:
photo = photo[((photo['label'] == 'food') | (photo['label'] == 'drink' )) & (photo['caption'] != '') ]
photo.to_csv('photo_captions.csv',index=False)

In [ ]:
###